In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import boto3
import os

In [2]:
name = []
mileage = []
dealer_name = []
rating = []
rating_count = []
price = []

for i in range(1,41):
    
    # Website in a variable
    website = 'https://www.cars.com/shopping/results/?page=' + str(i) + '&page_size=20&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=bmw&maximum_distance=all&mileage_max=&models[]=bmw-335&sort=best_match_desc&stock_type=used&year_max=2011&year_min=2007&zip='

    # Request to website
    response = requests.get(website)
    
    # Soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Results
    results = soup.find_all('div', {'class':'vehicle-card'})
    
    
    for result in results:
    
        # name
        try:
            name.append(result.find('h2').get_text())
        except:
            name.append('n/a')


        # mileage
        try:
            mileage.append(result.find('div', {'class':'mileage'}).get_text())
        except:
            mileage.append('n/a')

        # dealer_name
        try:
            dealer_name.append(result.find('div', {'class': 'dealer-name'}).get_text().strip())
        except:
            dealer_name.append('n/a')

        # rating
        try:
            rating.append(result.find('span',{'class':'sds-rating__count'}).get_text())
        except:
            rating.append('n/a')


        # rating_count
        try:
            rating_count.append(result.find('span', {'class': 'sds-rating__link'}).get_text())
        except:
            rating_count.append('n/a')


        # price
        try:
            price.append(result.find('span', {'class': 'primary-price'}).get_text())
        except:
            price.append('n/a')

In [3]:
bmw_df = pd.DataFrame({'Name':name, 'Mileage': mileage,'Dealer Name': dealer_name,
                          'Rating': rating, 'Rating Count': rating_count, 'Price':price})

In [4]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price
0,2011 BMW 335 i xDrive,"74,555 mi.",O'Meara Volkswagen of Thornton,4.5,(705 reviews),"$17,511"
1,2009 BMW 335 i,"82,481 mi.",Caspian Auto Motors,3.7,(170 reviews),"$14,379"
2,2011 BMW 335 i xDrive,"61,618 mi.","Guess Buick GMC / Guess Ford, Inc.",4.9,(74 reviews),"$18,200"
3,2010 BMW 335 i,"59,483 mi.",Carvana-Touchless Delivery To Your Home,4.2,"(8,906 reviews)","$20,990"
4,2011 BMW 335 i,"46,681 mi.",Gulf Coast Auto Brokers of Sarasota,4.1,(84 reviews),"$20,995"
...,...,...,...,...,...,...
356,2009 BMW 335 i,"91,121 mi.",PDX Motors,3.7,(16 reviews),"$15,991"
357,2008 BMW 335 i,"93,321 mi.",Precise Automotive Group,2.1,(20 reviews),"$15,899"
358,2009 BMW 335 i,"56,114 mi.",Norm Reeves Honda Superstore North Richland Hills,3.7,(3 reviews),"$17,619"
359,2010 BMW 335 i,"104,367 mi.",Apexcars.net,4.7,(26 reviews),"$16,999"


### Data Cleaning

In [5]:
bmw_df['Rating Count'] = bmw_df['Rating Count'].apply(lambda x: x.strip('reviews)').strip('('))

# JUNE 2nd, 2022
## Further data cleaning prep for SQL

### Parse year

In [6]:
bmw_df['Year'] = bmw_df['Name'].apply(lambda x: x.split(' ')[0])

In [7]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Year
0,2011 BMW 335 i xDrive,"74,555 mi.",O'Meara Volkswagen of Thornton,4.5,705,"$17,511",2011
1,2009 BMW 335 i,"82,481 mi.",Caspian Auto Motors,3.7,170,"$14,379",2009
2,2011 BMW 335 i xDrive,"61,618 mi.","Guess Buick GMC / Guess Ford, Inc.",4.9,74,"$18,200",2011
3,2010 BMW 335 i,"59,483 mi.",Carvana-Touchless Delivery To Your Home,4.2,"8,906","$20,990",2010
4,2011 BMW 335 i,"46,681 mi.",Gulf Coast Auto Brokers of Sarasota,4.1,84,"$20,995",2011
...,...,...,...,...,...,...,...
356,2009 BMW 335 i,"91,121 mi.",PDX Motors,3.7,16,"$15,991",2009
357,2008 BMW 335 i,"93,321 mi.",Precise Automotive Group,2.1,20,"$15,899",2008
358,2009 BMW 335 i,"56,114 mi.",Norm Reeves Honda Superstore North Richland Hills,3.7,3,"$17,619",2009
359,2010 BMW 335 i,"104,367 mi.",Apexcars.net,4.7,26,"$16,999",2010


### Remove 'mi' from mileage

In [8]:
bmw_df['Mileage'] = bmw_df['Mileage'].apply(lambda x: x.split(' ')[0])

### Remove \$ from price


In [9]:
'''
Reasoning for the -1 for future reference:
because if happens that occurrence is not in the string you'll get "IndexError: list index out of range".

Therefore -1 will not get any harm cause number of occurrences is already set to one.
'''

bmw_df['Price'] = bmw_df['Price'].apply(lambda x: x.split('$')[-1])


In [10]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Year
0,2011 BMW 335 i xDrive,"74,555",O'Meara Volkswagen of Thornton,4.5,705,"17,511",2011
1,2009 BMW 335 i,"82,481",Caspian Auto Motors,3.7,170,"14,379",2009
2,2011 BMW 335 i xDrive,"61,618","Guess Buick GMC / Guess Ford, Inc.",4.9,74,"18,200",2011
3,2010 BMW 335 i,"59,483",Carvana-Touchless Delivery To Your Home,4.2,"8,906","20,990",2010
4,2011 BMW 335 i,"46,681",Gulf Coast Auto Brokers of Sarasota,4.1,84,"20,995",2011
...,...,...,...,...,...,...,...
356,2009 BMW 335 i,"91,121",PDX Motors,3.7,16,"15,991",2009
357,2008 BMW 335 i,"93,321",Precise Automotive Group,2.1,20,"15,899",2008
358,2009 BMW 335 i,"56,114",Norm Reeves Honda Superstore North Richland Hills,3.7,3,"17,619",2009
359,2010 BMW 335 i,"104,367",Apexcars.net,4.7,26,"16,999",2010


### Remove comas from Mileage and Price

In [11]:
bmw_df['Mileage'] = bmw_df['Mileage'].apply(lambda x: x.replace(',', ''))

In [12]:
bmw_df['Price'] = bmw_df['Price'].apply(lambda x: x.replace(',', ''))

## Save CSV 

In [13]:
filename = 'data_scrapped_' + datetime.today().strftime('%Y-%m-%d')
# bmw_df.to_csv(filename, index=False)

## Upload to S3

In [14]:
from secrets import access_key, secret_access_key

client = boto3.client('s3',
                     aws_access_key_id = access_key,
                     aws_secret_access_key = secret_access_key)

In [22]:
upload_file_bucket = 'car-scrapper-bucket'
upload_file_key = 'csv/' + str(filename)

client.upload_file(filename, upload_file_bucket, upload_file_key)
